In [3]:
# pip install -r requirements.txt
import numpy as np
import pandas as pd
import os

In [17]:
streaming_pre = './streaming_history_files'
streaming_paths = os.listdir(streaming_pre)
audio_paths = [x for x in streaming_paths if "Audio" in x]
audio_2025_paths = [x for x in audio_paths if "2025" in x]
audio_2025_paths

['Streaming_History_Audio_2025_13.json',
 'Streaming_History_Audio_2024-2025_12.json',
 'Streaming_History_Audio_2025_14.json']

## 2025 Streams

In [104]:
audio_2025_df_og = pd.DataFrame()
for f in sorted(audio_2025_paths):
  df_temp = pd.read_json(streaming_pre+"/"+f)
  df_temp["stream_date"] = pd.to_datetime(df_temp["ts"]).dt.date
  audio_2025_df_og = pd.concat([audio_2025_df_og, df_temp])
audio_2025_df_og = audio_2025_df_og[(audio_2025_df_og["stream_date"]>=pd.to_datetime("2025-01-01").date()) & 
                              (audio_2025_df_og["stream_date"]<=pd.to_datetime("2025-12-31").date())]
audio_2025_df_og = audio_2025_df_og[~audio_2025_df_og["master_metadata_track_name"].isna()]
print(audio_2025_df_og["stream_date"].min())
print(audio_2025_df_og["stream_date"].max())
print(audio_2025_df_og.shape)



2025-01-01
2025-10-09
(22921, 24)


In [126]:
audio_2025_df = audio_2025_df_og.copy()
audio_2025_df = audio_2025_df_og[['platform', 'ms_played', 'master_metadata_track_name', 'master_metadata_album_artist_name', 
                               'master_metadata_album_album_name', 'spotify_track_uri', 'reason_start', 'reason_end', 
                               'skipped', 'stream_date']]
audio_2025_df["seconds"] = audio_2025_df["ms_played"]/1000
audio_2025_df["minutes"] = audio_2025_df["seconds"]/60
audio_2025_df.head()

<ipython-input-126-0ea1d0538d8e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  audio_2025_df["seconds"] = audio_2025_df["ms_played"]/1000
<ipython-input-126-0ea1d0538d8e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  audio_2025_df["minutes"] = audio_2025_df["seconds"]/60


,platform,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,reason_start,reason_end,skipped,stream_date,seconds,minutes
10278,ios,219813,Sunshine Baby,The Japanese House,In the End It Always Does,spotify:track:7ihVeKd4RjiZ3y12nDhPZO,trackdone,trackdone,False,2025-01-01,219.813,3.663550
10279,ios,168390,How Long,Yana,How Long,spotify:track:4Nj7olHNFww7pswTjw5K69,trackdone,fwdbtn,True,2025-01-01,168.390,2.806500
10280,ios,23120,If Now Was Then,Maggie Rogers,Don't Forget Me,spotify:track:4FQiOvG6qN55tdzvu37uUN,fwdbtn,endplay,True,2025-01-01,23.120,0.385333
10281,ios,267653,The Only Exception,Paramore,Brand New Eyes,spotify:track:7JIuqL4ZqkpfGKQhYlrirs,clickrow,endplay,True,2025-01-01,267.653,4.460883
10282,ios,261960,Decode,Paramore,Decode,spotify:track:1ZLtE9tSJdaUiIJ9YoKHQe,trackdone,trackdone,False,2025-01-01,261.960,4.366000


In [127]:
minutes_played = audio_2025_df["seconds"].sum()/60
songs_played = audio_2025_df["spotify_track_uri"].count()
unique_songs_played = audio_2025_df.groupby(["master_metadata_track_name", "master_metadata_album_artist_name"]).ngroups
print(minutes_played)
print(songs_played)
print(unique_songs_played)

38921.6226
22921
5178


In [134]:
most_played_songs = audio_2025_df.groupby(["master_metadata_track_name", "master_metadata_album_artist_name"])["master_metadata_track_name"]\
                         .count().reset_index(name="plays").sort_values("plays", ascending=False)
print(most_played_songs["plays"].sum())
print("Most played songs: ")
most_played_songs.head(5)

22921
Most played songs: 


,master_metadata_track_name,master_metadata_album_artist_name,plays
2858,Nettles,Ethel Cain,104
2108,Illegal,PinkPantheress,89
4322,Tonight,PinkPantheress,86
4152,The Edge,Sydney Ross Mitchell,81
5050,party 4 u,Charli xcx,79


In [139]:
longest_played_songs = audio_2025_df.groupby(["master_metadata_track_name", "master_metadata_album_artist_name"])["minutes"]\
                         .sum().reset_index(name="minutes").sort_values("minutes", ascending=False)
print(longest_played_songs["minutes"].sum())
print("Longest played songs: ")
longest_played_songs.head(5)

38921.622599999995
Longest played songs: 


,master_metadata_track_name,master_metadata_album_artist_name,minutes
2858,Nettles,Ethel Cain,698.383150
1528,Fuck Me Eyes,Ethel Cain,351.318450
4152,The Edge,Sydney Ross Mitchell,251.258967
5050,party 4 u,Charli xcx,245.392267
3974,Sunshine Baby,The Japanese House,204.949717


In [135]:
most_played_artists = audio_2025_df.groupby(["master_metadata_album_artist_name"])["master_metadata_track_name"]\
                         .count().reset_index(name="plays").sort_values("plays", ascending=False)
print(most_played_artists["plays"].sum())
print("Most played artists: ")
most_played_artists.head(5)

22921
Most played artists: 


,master_metadata_album_artist_name,plays
303,Charli xcx,728
1720,Zara Larsson,641
516,Ethel Cain,586
334,Clairo,584
1247,PinkPantheress,544


In [136]:
longest_played_artists = audio_2025_df.groupby(["master_metadata_album_artist_name"])["minutes"]\
                         .sum().reset_index(name="minutes").sort_values("minutes", ascending=False)
print(longest_played_artists["minutes"].sum())
print("Longest played artists: ")
longest_played_artists.head(5)

38921.6226
Longest played artists: 


,master_metadata_album_artist_name,minutes
516,Ethel Cain,2103.026683
303,Charli xcx,1342.694650
334,Clairo,1245.126350
1720,Zara Larsson,1208.443650
1543,The Marías,1066.552733


## All-Time Streaming